In [1]:
import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

C:\Users\Abhijit\Documents\GitHub\cpsc490\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(
        num_val=0.05, 
        num_test=0.1, 
        is_undirected=True, 
        add_negative_train_samples=False
    ),
])

dataset = Planetoid("../data/Cora", name='Cora', transform=transform)

# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.

train_data, val_data, test_data = dataset[0]

In [3]:
from torch import nn
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        
        # self.W1 = nn.Linear(out_channels * 2, out_channels)
        # self.W2 = nn.Linear(out_channels, 1)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        # z = torch.cat((z[edge_label_index[0]], z[edge_label_index[1]]), dim=1)
        # return self.W2(F.relu(self.W1(z)).squeeze())
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [4]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 201):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Epoch: 010, Loss: 0.6896, Val: 0.7359, Test: 0.7443
Epoch: 020, Loss: 0.6356, Val: 0.7746, Test: 0.7566
Epoch: 030, Loss: 0.5678, Val: 0.7890, Test: 0.7659
Epoch: 040, Loss: 0.5294, Val: 0.8271, Test: 0.8149
Epoch: 050, Loss: 0.4920, Val: 0.8598, Test: 0.8724
Epoch: 060, Loss: 0.4764, Val: 0.8680, Test: 0.8903
Epoch: 070, Loss: 0.4601, Val: 0.8709, Test: 0.9002
Epoch: 080, Loss: 0.4508, Val: 0.8761, Test: 0.9033
Epoch: 090, Loss: 0.4461, Val: 0.8961, Test: 0.9085
Epoch: 100, Loss: 0.4506, Val: 0.8922, Test: 0.9124
Epoch: 110, Loss: 0.4428, Val: 0.8981, Test: 0.9140
Epoch: 120, Loss: 0.4411, Val: 0.8978, Test: 0.9166
Epoch: 130, Loss: 0.4376, Val: 0.8983, Test: 0.9172
Epoch: 140, Loss: 0.4256, Val: 0.9068, Test: 0.9208
Epoch: 150, Loss: 0.4249, Val: 0.9116, Test: 0.9229
Epoch: 160, Loss: 0.4263, Val: 0.9131, Test: 0.9233
Epoch: 170, Loss: 0.4201, Val: 0.9125, Test: 0.9243
Epoch: 180, Loss: 0.4204, Val: 0.9083, Test: 0.9224
Epoch: 190, Loss: 0.4118, Val: 0.9091, Test: 0.9227
Epoch: 200, 